In [1]:
import simpy
import random
import numpy as np

RNG = np.random.default_rng(seed=42)

In [ ]:
class OpenQueueNetwork:
    def __init__(self, lambda_, mu_cpu=10, mu_fast=12, mu_slow=9):
        self.env = simpy.Environment()

        # Arrival and service parameters
        self.beta_arr = 1 / lambda_
        self.beta_cpu = 1 / mu_cpu
        self.beta_fast = 1 / mu_fast
        self.beta_slow = 1 / mu_slow

        # Resources
        self.cpu = simpy.Resource(self.env, capacity=1)
        self.fast_disk = simpy.Resource(self.env, capacity=1)
        self.slow_disk = simpy.Resource(self.env, capacity=1)

        # Metrics
        self.completed_jobs = 0
        self.response_times = []

    def cpu_process(self, job_id, arrival_time):
        with self.cpu.request() as req:
            yield req
            yield self.env.timeout(RNG.exponential(self.beta_cpu))

        # Route to disk (to be defined per case)
        yield from self.disk_routing(job_id, arrival_time)

    def fast_disk_process(self, job_id, arrival_time):
        with self.fast_disk.request() as req:
            yield req
            yield self.env.timeout(RNG.exponential(self.beta_fast))

        self.completed_jobs += 1
        self.response_times.append(self.env.now - arrival_time)

    def slow_disk_process(self, job_id, arrival_time):
        with self.slow_disk.request() as req:
            yield req
            yield self.env.timeout(RNG.exponential(self.beta_slow))

        self.completed_jobs += 1
        self.response_times.append(self.env.now - arrival_time)

    def job_generator(self):
        job_id = 0
        while True:
            yield self.env.timeout(RNG.exponential(self.beta_arr))
            arrival_time = self.env.now
            self.env.process(self.cpu_process(job_id, arrival_time))
            job_id += 1

    def run(self, sim_time):
        self.env.process(self.job_generator())
        self.env.run(until=sim_time)